In [ ]:
import sagemaker

In [ ]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [ ]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

In [ ]:
model_data = "s3://{0}/stable-diffusion-webui/assets/model.tar.gz".format(bucket)
model_name = None
image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/all-in-one-ai-stable-diffusion-webui-inference:latest'.format(account_id, region_name)
model_environment = {
    'embeddings_s3uri': 's3://{0}/stable-diffusion-webui/embeddings/'.format(bucket), 
    'hypernetwork_s3uri': 's3://{0}/stable-diffusion-webui/hypernetwork/'.format(bucket), 
    'api_endpoint': 'noapi',
    'ckpt': '/opt/ml/model/768-v-ema.ckpt',
    'SAGEMAKER_MODEL_SERVER_TIMEOUT': '1200'
}

In [ ]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

In [ ]:
model = Model(
    name = model_name,
    model_data = model_data,
    role = role,
    image_uri = image_uri,
    env = model_environment,
    predictor_cls = Predictor
)

In [ ]:
from sagemaker.async_inference import AsyncInferenceConfig
endpoint_name = None
instance_type = 'ml.g4dn.2xlarge'
instance_count = 1
async_config = AsyncInferenceConfig(output_path='s3://{0}/{1}/asyncinvoke/out/'.format(bucket, 'stable-diffusion-webui'))

In [ ]:
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    async_inference_config = async_config
)

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

inputs = {
    'task': 'text-to-image', 
    'txt2img_payload': {
        'enable_hr': False, 
        'denoising_strength': 0.7, 
        'firstphase_width': 0, 
        'firstphase_height': 0, 
        'prompt': 'girl', 
        'styles': ['None', 'None'], 
        'seed': -1.0, 
        'subseed': -1.0, 
        'subseed_strength': 0, 
        'seed_resize_from_h': 0, 
        'seed_resize_from_w': 0, 
        'sampler_index': 'Euler a', 
        'batch_size': 1, 
        'n_iter': 1, 
        'steps': 20, 
        'cfg_scale': 7, 
        'width': 768, 
        'height': 768, 
        'restore_faces': False, 
        'tiling': False, 
        'negative_prompt': '', 
        'eta': 1, 
        's_churn': 0, 
        's_tmax': None, 
        's_tmin': 0, 
        's_noise': 1, 
        'override_settings': {}, 
        'script_args': '[0, false, false, false, "", 1, "", 0, "", true, false, false]'}, 
        'username': ''
}

prediction = predictor.predict_async(inputs)

In [ ]:
from sagemaker.async_inference.waiter_config import WaiterConfig
print(f"Response object: {prediction}")
print(f"Response output path: {prediction.output_path}")
print("Start Polling to get response:")

import time

start = time.time()

config = WaiterConfig(
  max_attempts=100, #  number of attempts
  delay=10 #  time in seconds to wait between attempts
  )

prediction.get_result(config)

print(f"Time taken: {time.time() - start}s")

In [ ]:
import json
import io
from PIL import Image
s3_resource = boto3.resource('s3')

def get_bucket_and_key(s3uri):
    pos = s3uri.find('/', 5)
    bucket = s3uri[5 : pos]
    key = s3uri[pos + 1 : ]
    return bucket, key

In [ ]:
import traceback
from PIL import Image
import base64
try:
    bucket, key = get_bucket_and_key(prediction.output_path)
    obj = s3_resource.Object(bucket, key)
    body = obj.get()['Body'].read().decode('utf-8') 
    for image in json.loads(body)['images']:
        image = Image.open(io.BytesIO(base64.b64decode(image)))
        image.show()
except Exception as e:
    traceback.print_exc()
    print(e)

In [ ]:
predictor.delete_endpoint()

In [ ]:
import sagemaker
from sagemaker.estimator import Estimator

In [ ]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [ ]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

In [ ]:
import json
images_s3uri = 's3://{0}/stable-diffusion-webui/images/'.format(bucket)
models_s3uri = 's3://{0}/stable-diffusion-webui/models/'.format(bucket)
embeddings_s3uri = 's3://{0}/stable-diffusion-webui/embeddings/'.format(bucket)
hypernetwork_s3uri = 's3://{0}/stable-diffusion-webui/hypernetwork/'.format(bucket)

train_args = {
  'embedding_settings': {
    'name': 'aws-puppet',
    'nvpt': 6,
    'overwrite_old': True,
    'initialization_text': '*'
  },
  'images_preprocessing_settings': {
    'process_width': 512,
    'process_height': 512,
    'preprocess_txt_action': 'ignore',
    'process_flip': True,
    'process_split': False,
    'process_caption': False,
    'process_caption_deepbooru': True,
    'process_split_threshold': 0.5,
    'process_overlap_ratio': 0.2,
    'process_focal_crop': False,
    'process_focal_crop_face_weight': 0.9,
    'process_focal_crop_entropy_weight': 0.15,
    'process_focal_crop_edges_weight': 0.5,
    'process_focal_crop_debug': False
  },
  'train_embedding_settings': {
    'learn_rate': '0.005',
    'batch_size': 1,
    'gradient_step': 1,
    'training_width': 512,
    'training_height': 512,
    'steps': 30000,
    'shuffle_tags': False,
    'tag_drop_out': 0,
    'latent_sampling_method': 'once',
    'create_image_every': 500,
    'save_embedding_every': 500,
    'save_image_with_stored_embedding': True,
    'preview_from_txt2img': False,
    'txt2img_preview_params': [
      'girl',
      '',
      20,
      0,
      7,
      -1.0,
      512,
      512
    ]
  }
}

def s3uri_contain_files(s3uri):
  s3_client = boto3.client('s3')
  s3_resource= boto3.resource('s3')
  bucket, key = get_bucket_and_key(s3uri)
  s3_bucket = s3_resource.Bucket(bucket)
  objs = list(s3_bucket.objects.filter(Prefix=key))
  return len(objs) > 1

def json_encode_hyperparameters(hyperparameters):
    for (k, v) in hyperparameters.items():
        print(k, v)
    
    return {str(k): json.dumps(v) for (k, v) in hyperparameters.items()}

hyperparameters = {
    'train-args': json.dumps(train_args),
    'train-task': 'embedding',
    'embeddings-s3uri': embeddings_s3uri,
    'hypernetwork-s3uri': hypernetwork_s3uri,
    'region': region_name,
    'api-endpoint': 'noapi'
}

hyperparameters = json_encode_hyperparameters(hyperparameters)

image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/all-in-one-ai-stable-diffusion-webui-training'.format(account_id, region_name)
instance_type = 'ml.g4dn.xlarge'
instance_count = 1
with_embeddings = s3uri_contain_files(embeddings_s3uri)
with_hypernetwork = s3uri_contain_files(hypernetwork_s3uri)

inputs = {
    'images': images_s3uri,
    'models': models_s3uri
}

if with_embeddings:
  inputs['embeddings'] = embeddings_s3uri
if with_hypernetwork:
  inputs['hypernetwork'] = hypernetwork_s3uri

In [ ]:
print(images_s3uri)
print(models_s3uri)
# Please exectute tools/prepare.py with the path to your model files directory.
!aws s3 cp images/ $images_s3uri --recursive

In [ ]:
estimator = Estimator(
    role = role,
    instance_count=1,
    instance_type = instance_type,
    image_uri = image_uri,
    hyperparameters = hyperparameters
)

In [ ]:
estimator.fit(inputs)